In [1]:
import pandas as pd
import ib_insync
from ib_insync import *
import logging
import time
from yfinance import Ticker
import yfinance as yf

# Set up logging to log real-time trade updates
logging.basicConfig(filename='rt_trades.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# create logger
logging.info('start log')

In [2]:
util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)

ib = IB()
ib.disconnect()

# ib.connect(clientId=1)
ib.connect(host='127.0.0.1', port=7497, clientId=6)

<IB connected to 127.0.0.1:7497 clientId=6>

In [3]:
# init dataframe
df = pd.DataFrame(columns=['date', 'last', 'vol'])
df.set_index('date', inplace=True)


def new_data(tickers):
    ''' process incoming data and check for trade entry '''
    
    global df

    for ticker in tickers:
        df.loc[ticker.time] = [ticker.last, ticker.volume]
        current_volume = ticker.volume
        get_52_week_low(ticker.contract.symbol)
        print(f"Symbol: {ticker.contract.symbol}, Volume: {current_volume}")
        

    # Get current date and ex-dividend date from dividend history
    today = datetime.datetime.today().date()
    ex_dividend_date = dividends.index[-1].date()  # Getting the most recent ex-dividend date

    print(f"Today's Date: {today}")
    print(f"Ex-Dividend Date: {ex_dividend_date}")

    if today < ex_dividend_date:
        print(f"Buying {symbol} stock before Ex-Dividend Date")    
        
        five_mins_ago = df.index[-1] - pd.Timedelta(minutes=2)
        if df.index[0] < five_mins_ago:
            df = df[five_mins_ago:]

            #pricing analysis
            price_min = df['last'].min()
            print("price_min =",price_min)
            logging.info('price_min' + str(price_min))
            price_max = df['last'].max()
            print("price_max =",price_max)
            logging.info('price_max' + str(price_max))

            #volume analysis
            vol_min = df['vol'].min()
            print("vol_min =",vol_min)
            vol_max = df['vol'].max()
            print("vol_max =",vol_max)
            '''
            If the current price is more than 5% higher than the lowest price over 
            the last 5 minutes, we will execute a buy order. Otherwise, 
            if the current price is more than 5% lower than the highest price 
            in the past 5 minutes, we will send a sell order.
            '''
            if df['last'].iloc[-1] > price_min * 1.01:
                submit_order('BUY')
                print('BUY ==', df['last'].iloc[-1])
                logging.info('BUY ==' + df['last'].iloc[-1])
            ''' elif df['last'].iloc[-1] < price_max * 0.599:
                submit_order('SELL')
                print('SELL ==', df['last'].iloc[-1])
                logging.info('SELL ==' + df['last'].iloc[-1])
                '''

            # If we bought before the ex-dividend date, now plan to sell after the payout date
            # You should check the dividend payout date and wait for it before selling
            dividend_payout_date = ex_dividend_date + datetime.timedelta(days=1)  # Assume 1 days after ex-dividend

            if today >= dividend_payout_date:
                print(f"Selling {symbol} stock after dividend payout")

                # Create sell order (market order for simplicity)
                if df['last'].iloc[-1] < price_max * 0.599:
                    submit_order('SELL')
                    print('SELL ==', df['last'].iloc[-1])
        else:
            print(f"Already passed Ex-Dividend Date, waiting for next opportunity.")            


def submit_order(direction):
    ''' place order with IB - exit if order gets filled '''
    generic_order = MarketOrder(direction, 5)
    trade = ib.placeOrder(generic_contract, generic_order)
    ib.sleep(3)
    print(f"Order status: {trade.orderStatus.status}")
    if trade.orderStatus.status == 'Filled':
        ib.disconnect()
        quit()

def get_52_week_low(symbolX):
    try:
        time.sleep(12)  # Avoid rate limits
        stock = Ticker(symbolX)
        low_52_week = stock.info['fiftyTwoWeekLow']
        print(f"52-week low for {symbolX}: {low_52_week}")
        return low_52_week
    except Exception as e:
        print(f"Error fetching 52-week low for {symbolX}: {e}")
        return None

# Function to log trade updates
def log_trade_update(trade, fill):
    logging.info(f"Trade Updated: {trade}, Fill Price: {fill.price}, Quantity: {fill.shares}, "
                 f"Order Status: {trade.status}")
    
# Function to handle account summary updates
def handle_account_summary(account_summary):
    # Extract the tag, value, and currency
    if account_summary.tag == 'CashBalance':
        print(f"Tag: {account_summary.tag}, Value: {account_summary.value}, "
              f"Currency: {account_summary.currency}")
        logging.info(f"Tag: {account_summary.tag}, Value: {account_summary.value}, "
                     f"Currency: {account_summary.currency}")


# Create contracts
tick = 'F'
stock = yf.Ticker(tick)
dividends = stock.dividends
# Save data to a CSV file to avoid repeated requests
dividends.to_csv(f'dividends_{tick}.csv')
print(dividends)
generic_contract = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(generic_contract)

# Request market data for generic stock
ib.reqMarketDataType(3)
ib.reqMktData(generic_contract)

# Set callback function for tick data
ib.pendingTickersEvent += new_data
ib.orderStatusEvent += log_trade_update
ib.accountSummaryEvent += handle_account_summary # Subscribe to account summary updates

# Request the account summary to begin receiving updates
tags = ['BuyingPower', 'CashBalance', 'NetLiquidation']  # Tags to request
ib.reqAccountSummary()

# Run infinitely
ib.run()

YFRateLimitError: Too Many Requests. Rate limited. Try after a while.